In [ ]:
pip install gradio

In [8]:
import gradio as gr
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Pycaret to train various ML models
from pycaret.classification import *

In [9]:
import numpy
numpy.version.version

'1.26.2'

In [10]:
numpy.__version__

'1.26.2'

In [11]:
def encode_ages(df): # Binning ages 
    df.Age = df.Age.fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    categories = pd.cut(df.Age, bins, labels=False)
    df.Age = categories
    return df

def encode_fares(df): # Binning fares
    df.Fare = df.Fare.fillna(-0.5)
    bins = (-1, 0, 8, 15, 31, 1000)
    categories = pd.cut(df.Fare, bins, labels=False)
    df.Fare = categories
    return df

def encode_sex(df):
    mapping = {"male": 0, "female": 1}
    return df.replace({'Sex': mapping})

In [12]:
import warnings
warnings.filterwarnings('ignore')


def transform_features(df):
    df = encode_ages(df)
    df = encode_fares(df)
    df = encode_sex(df)
    return df

# Asyraf added
  # train = data[['PassengerId', 'Fare', 'Age', 'Sex', 'Survived']]
  # train = transform_features(train)


  # X_all = train.drop(['Survived', 'PassengerId'], axis=1)
  # y_all = train['Survived']

  # num_test = 0.20
  # X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

  # # Pass what model to train
  # # if model== 'RF':
  # clf = RandomForestClassifier()
  # clf.fit(X_train, y_train)
  # predictions = clf.predict(X_test)
  # acc = accuracy_score(y_test, predictions)
  # print("accuracy of Random Forest: ", acc)
  # elif model == 'LR':
  #   clf = RandomForestClassifier()
  #   clf.fit(X_train, y_train)
  #   predictions = clf.predict(X_test)
  #   acc = accuracy_score(y_test, predictions)
  #   print("accuracy of Logistic Regression: ", acc)
  # else:
  #   pass

  # return {'Accuracy of Random Forest': int(acc)}

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/gradio-app/titanic/master/train.csv')

# data = transform_features(data[['PassengerId', 'Fare', 'Age', 'Sex', 'Survived']])
# y_out = data['Survived']
s = setup(data, target = 'Survived',feature_selection = True, ignore_features = ['Cabin', 'Parch', 'Ticket', 'SibSp', 'Embarked', 'Name', 'Pclass'])

best = compare_models(n_select = 15)
compare_model_results = pull()


# If want to do manual split train testing 
# train = data[['PassengerId', 'Fare', 'Age', 'Sex', 'Survived']]
# train = transform_features(train)

# X_all = train.drop(['Survived', 'PassengerId'], axis=1)
# y_all = train['Survived']
# # print('yalll is:',y_all)

# num_test = 0.2
# X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

### Write a prediction function

In [14]:
# Modify fn ,add can pick what model to train on and pass accuracy as well - Asyraf
def predict_survival(sex, age, fare, model):

  df = pd.DataFrame.from_dict({'Sex': [sex], 'Age': [age], 'Fare': [fare]})
  df = encode_sex(df)
  df = encode_fares(df)
  df = encode_ages(df)

  model_index = list(compare_model_results['Model']).index(model)
  model = best[model_index]

  # pred = predict_model(model, df,raw_score=True)
  pred = model.predict_proba(df)[0]
  
  return {'Perishes': pred[0].astype('float64'), 'Survives': pred[1].astype('float64')}

### Build a shareable UI in 4 lines of code!

In [15]:
sex = gr.inputs.Radio(['female', 'male'], label="Sex")
age = gr.inputs.Slider(minimum=0, maximum=120, default=22, label="Age")
fare = gr.inputs.Slider(minimum=0, maximum=1000, default=100, label="Fare (british pounds)")
# Add radio button to train and can choose what classifier to train
model = gr.inputs.Dropdown(list(compare_model_results['Model']), label="Choose Machine Learning Classifier")

gr.Interface(predict_survival, [sex, age, fare, model], "label", live=True).launch(debug=True);

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\aicyb\anaconda3\lib\site-packages\gradio\routes.py", line 414, in run_predict
    output = await app.get_blocks().process_api(
  File "C:\Users\aicyb\anaconda3\lib\site-packages\gradio\blocks.py", line 1323, in process_api
    result = await self.call_function(
  File "C:\Users\aicyb\anaconda3\lib\site-packages\gradio\blocks.py", line 1051, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\aicyb\anaconda3\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "C:\Users\aicyb\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "C:\Users\aicyb\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 807, in run
    result = context.run(func, *args)
  File "C:\Users\aicyb\AppData\Local\Temp\ipykernel_14144\1596690375.py", line 9, in predict_survival
    m

Keyboard interruption in main thread... closing server.
